In [ ]:
# ===========================================================================
# DIANI Mamoudou Sékou
#
# Fevrier 2022
#===========================================================================

In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
from sklearn import preprocessing
import numpy as np
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB

In [ ]:
# Recupération des données situées dans "C:/Users/diani/Desktop/Apprntissage_auto/DataPrepa1.csv"
# lecture du CSV et affichage des 5 premieres lignes
Pollution_data = pd.read_csv('C:/Users/diani/Desktop/Apprntissage_auto/DataPrepa1.csv', sep = ';', index_col="id")
pd.set_option('display.max_columns', None)
Pollution_data.head(5)

In [ ]:
# Normalisation des donnees avec la normalisation MinMax parce que pour determiner les 7 meilleures features avec chi2
# nous devons avoir des valeurs positives
features = ["tLengthOfScreenName", "tLengthOfDescriptionInUserProfile", "tLongevityOfTheAccount", "tNumerOfFollowings", "tNumberOfFollowers", "tSeriesOfNumberOfFollowings", "Following.followers", "tNumberOfTweets", "tNumberOfTweetsByDays", "Tweets.theLongevityOfTheAccount", "TweetURL.tNumberOfTweets", "tNbMoyenUrlByTweet", "UsernameInTweet.tNumberOfTweets", "TimeMoyenBet2ConseTweet", "ValueOfTimeMaxBet2ConseTweet"]
std_MinMax = preprocessing.MinMaxScaler().fit(Pollution_data[features])
normalized_data = std_MinMax.transform(Pollution_data[features])
labels = np.array(Pollution_data['Status'])

In [ ]:
# Recherche des 7 meilleures features avec Chi2
chi_scores = chi2(normalized_data, labels)


# Recupération des donnees et tries du plus imortant au moins important
chi_info = []
for i in range (len(chi_scores[0])):
    chi_info.append([features[i], chi_scores[0][i]])
chi_out = sorted(chi_info, key = lambda x : x[1], reverse=True)


#Affichage des 7 meilleurs features et de leurs scores chi2 
liste_newFeatures_chi2=[]
for i in range (7):
    x=chi_out[i]
    liste_newFeatures_chi2.append(x[0])
print(liste_newFeatures_chi2)
position=[]
for i in range (len(liste_newFeatures_chi2)):
    for j in range (len(features)):
        if liste_newFeatures_chi2[i] == features[j]:
            position.append(j)
#print (position)


### Elements du dataset à supprimer
l=[]
a=0
for i in range (len(features)):
    if i in position:
        a=a+1
    else:
        l.append(i)
print (l)



# Determination de la nouvelle liste de données
newList=[]
for i in range(len(normalized_data)):
    myArray = np.array(normalized_data[i])
    modifiedArray = np.delete(myArray, l)
    newList.append(modifiedArray)
normalized_data=newList
#print (normalized_data[0])

In [ ]:
# Separation des donnees en données d'entrenement et des données de test
train_features, test_features, train_labels, test_labels = train_test_split(normalized_data, labels, test_size = 0.2, random_state = 42)


In [ ]:
# Cette fonction permet de déterminer les metriques de performance 
# True Positive Rate(TPR), False Positive Rate(FPR) et l'aire sous la courbe ROC (Receiver Operating Characteristic)
def TPR_FPR(test_labels, predicted_labels):
    cnf_matrix = confusion_matrix(test_labels, predicted_labels)
    #print (cnf_matrix)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)

    TPR = TP/(TP+FN)
    FPR = FP/(FP+TN)

    fpr, tpr, _ = roc_curve(test_labels, predicted_labels, pos_label=1)
    aire = metrics.auc(fpr, tpr)
    return (TPR, FPR, aire)

In [ ]:
str_TPR="True Positive Rate: "
str_FPR="False Positive Rate: "
str_Aire="l'aire sous la courbe ROC: "

In [ ]:
# Arbre de décision: entrainement de l'arbre et prédiction sur les données de test
# Et determination de la F-mesure, du FPR, TPR et de l'aire sous la courbe
ad_classifier = DecisionTreeClassifier(random_state=42)
d_tree1 = ad_classifier.fit(train_features,train_labels)
predicted_labels = d_tree1.predict(test_features)

tpr, fpr, ad_auc  =TPR_FPR(test_labels, predicted_labels)
Val_ad_tpr=tpr[1]
Val_ad_fpr=fpr[1]
f1 = f1_score(test_labels, predicted_labels, average='binary', pos_label=1)
print (str_TPR, Val_ad_tpr)
print (str_FPR, Val_ad_fpr)
print (str_Aire, ad_auc)
f1 = f1_score(test_labels, predicted_labels, average='binary', pos_label=1)
print("F1-score of decision tree: ", f1)
# True Positive Rate:  0.9097206703910614
# False Positive Rate:  0.09333333333333334
# l'aire sous la courbe ROC:  0.908193668528864
# F1-score of decision tree: 0.9145231944288441

In [ ]:
# Random forrest: entrainement et prédiction sur les données de test
# Et determination de la F-mesure, du FPR, TPR et de l'aire sous la courbe
rnd_forest  = RandomForestClassifier()
rnd_forest.fit(train_features,train_labels)
predicted_labels = rnd_forest.predict(test_features)

tpr, fpr, rf_auc  =TPR_FPR(test_labels, predicted_labels)
Val_rf_tpr=tpr[1]
Val_rf_fpr=fpr[1]
f1 = f1_score(test_labels, predicted_labels, average='binary', pos_label=1)
print (str_TPR, Val_rf_tpr)
print (str_FPR, Val_rf_fpr)
print (str_Aire, rf_auc)
f1 = f1_score(test_labels, predicted_labels, average='binary', pos_label=1)
print("F1-score of random forrest: ", f1)
# True Positive Rate:  0.9459217877094972
# False Positive Rate:  0.06196078431372549
# l'aire sous la courbe ROC:  0.9419805016978857
# F1-score of random forrest: 0.9466621938946661

In [ ]:
# Bagging: entrainement et prédiction sur les données de test
# Et determination de la F-mesure, du FPR, TPR et de l'aire sous la courbe
bagging = BaggingClassifier()
bagging_classifier = bagging.fit(train_features,train_labels)
predicted_labels = bagging_classifier.predict(test_features)

tpr, fpr, bag_auc  =TPR_FPR(test_labels, predicted_labels)
Val_bag_tpr=tpr[1]
Val_bag_fpr=fpr[1]
print (str_TPR,Val_bag_tpr)
print (str_FPR,Val_bag_fpr)
print (str_Aire,bag_auc)
f1 = f1_score(test_labels, predicted_labels, average='binary', pos_label=1)
print("F1-score of bagging: ", f1)
# True Positive Rate:  0.9289385474860336
# False Positive Rate:  0.06248366013071895
# l'aire sous la courbe ROC:  0.9332274436776573
# F1-score of bagging: 0.9415204678362573

In [ ]:
# AdaBoost: entrainement et prédiction sur les données de test
# Et determination de la F-mesure, du FPR, TPR et de l'aire sous la courbe
adaB = AdaBoostClassifier()
adaB_classifier = adaB.fit(train_features,train_labels)
predicted_labels = adaB_classifier.predict(test_features)

tpr, fpr, adaB_auc  =TPR_FPR(test_labels, predicted_labels)
Val_adaB_tpr=tpr[1]
Val_adaB_fpr=fpr[1]
print (str_TPR,Val_adaB_tpr)
print (str_FPR,Val_adaB_fpr)
print (str_Aire,adaB_auc)
f1 = f1_score(test_labels, predicted_labels, average='binary', pos_label=1)
print("F1-score of adaBoost: ", f1)
# True Positive Rate:  0.9275977653631285
# False Positive Rate:  0.06222222222222222
# l'aire sous la courbe ROC:  0.9326877715704531
# F1-score of adaBoost: 0.9365974729241877

In [ ]:
# Classification avec Naive Bayes: entrainement et prédiction sur les données de test
# Et determination de la F-mesure, du FPR, TPR et de l'aire sous la courbe
bayes_class=GaussianNB()
bayes_classifier = bayes_class.fit(train_features,train_labels)
predicted_labels = bayes_classifier.predict(test_features)

tpr, fpr, nb_auc  =TPR_FPR(test_labels, predicted_labels)
Val_nb_tpr=tpr[1]
Val_nb_fpr=fpr[1]
print (str_TPR,Val_nb_tpr)
print (str_FPR,Val_nb_fpr)
print (str_Aire,nb_auc)
f1 = f1_score(test_labels, predicted_labels, average='binary', pos_label=1)
print("F1-score of Naive bayes: ", f1)
# True Positive Rate:  0.7211173184357542
# False Positive Rate:  0.10483660130718954
# l'aire sous la courbe ROC:  0.8081403585642823
# F1-score of Naive bayes: 0.7964951252622484